#  加载数据
把测试的数据分割成两部分，这样保证所有的算法在对比的时候是处于一个水平的

In [1]:
import pandas as pd

df_train = pd.read_csv("train_train.csv")
df_test = pd.read_csv("train_test.csv")
df_train.describe()

C:\Users\ljw\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4116: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


PassengerId    Survived      Pclass         Age       SibSp  \
count   700.000000  700.000000  700.000000  561.000000  700.000000   
mean    452.717143    0.375714    2.317143   29.515758    0.531429   
std     257.433014    0.484653    0.836912   14.415910    1.146188   
min       2.000000    0.000000    1.000000    0.420000    0.000000   
25%     227.750000    0.000000    2.000000         NaN    0.000000   
50%     458.500000    0.000000    3.000000         NaN    0.000000   
75%     676.250000    1.000000    3.000000         NaN    1.000000   
max     891.000000    1.000000    3.000000   74.000000    8.000000   

            Parch        Fare  
count  700.000000  700.000000  
mean     0.401429   30.791731  
std      0.830566   41.078193  
min      0.000000    0.000000  
25%      0.000000    7.895800  
50%      0.000000   14.456250  
75%      0.000000   31.275000  
max      6.000000  263.000000

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 12 columns):
PassengerId    700 non-null int64
Survived       700 non-null int64
Pclass         700 non-null int64
Name           700 non-null object
Sex            700 non-null object
Age            561 non-null float64
SibSp          700 non-null int64
Parch          700 non-null int64
Ticket         700 non-null object
Fare           700 non-null float64
Cabin          162 non-null object
Embarked       699 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 65.7+ KB


# 特征提取

在使用时发现pandas自带的get_dummies不是很好用(比如测试数据的值域范围和训练不一致，导致预测时特征维数和训练时不一致；当然也可能是我没有用好)， 我自己定义了几个函数如下:

In [2]:
def feature_one_code(dtfm, column, all_values):    
    for ev  in all_values:
        dtfm[column + "_" + str(ev)] = dtfm[column].apply(lambda x : int(x == ev))

def feature_range_code(dtfm , column, start, end, gap):
    prev = -1
    for v in range(start, end, gap):
        dtfm[column + "_" + str(v)] =  \
            dtfm[column].apply(lambda x : int(x > prev and x < v))
        prev = v

主要做特征工程，模型上选择 lr和决策树来看效果。

In [3]:

def evaluate(train_feature, test_feature, train_target, test_target):
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression(verbose=10)
    lr.fit(train_feature, train_target)
    print("lr score", lr.score(test_feature, test_target))
    
    
    print("-----")
    from sklearn.tree import DecisionTreeClassifier
    
    dtc = DecisionTreeClassifier()
    dtc.fit(train_feature, train_target)
    print("decision tree:", dtc.score(test_feature, test_target))

## 取两维特征

In [9]:
def extract_features_2(dtfm):
    dtfm_ret = dtfm.drop(["PassengerId", "Name", "Cabin", "Ticket"], axis = 1)  
    feature_one_code(dtfm_ret, "Sex", ["male", "female"])
    feature_range_code(dtfm_ret, "Age", 10, 90, 10)       
    dtfm_ret.drop(["SibSp","Parch","Fare" ,"Embarked", "Pclass","Age","Sex"], axis = 1, inplace=True)
    return  dtfm_ret

train_feature = extract_features_2(df_train)
test_feature = extract_features_2(df_test)
train_target = train_feature.Survived
train_feature.drop(["Survived"], axis = 1, inplace = True)

test_target = test_feature.Survived
test_feature.drop(["Survived"], axis = 1, inplace = True)

evaluate(train_feature, test_feature, train_target, test_target)

[LibLinear]lr score 0.79057591623
-----
decision tree: 0.780104712042


# 取三维特征
## sex, age, Pclass

In [6]:
def extract_features_3(dtfm):
    dtfm_ret = dtfm.drop(["PassengerId", "Name", "Cabin", "Ticket"], axis = 1)
    
    feature_one_code(dtfm_ret, "Sex", ["male", "female"])
    feature_range_code(dtfm_ret, "Age", 10, 90, 10)      
    feature_one_code(dtfm_ret, "Pclass", [2, 3, 1])
    dtfm_ret.drop(["SibSp","Parch","Fare" ,"Embarked", "Pclass","Age","Sex"], axis = 1, inplace=True)
    return  dtfm_ret

train_feature = extract_features_3(df_train)
test_feature = extract_features_3(df_test)
train_target = train_feature.Survived
train_feature.drop(["Survived"], axis = 1, inplace = True)

test_target = test_feature.Survived
test_feature.drop(["Survived"], axis = 1, inplace = True)

evaluate(train_feature, test_feature, train_target, test_target)

[LibLinear]lr score 0.801047120419
-----
decision tree: 0.748691099476


# 取四维特征
## sex, age, Pclass， Embarked

In [5]:
def extract_features_4(dtfm):
    
    dtfm_ret = dtfm.drop(["PassengerId", "Name", "Cabin", "Ticket"], axis = 1)
    
    feature_one_code(dtfm_ret, "Sex", ["male", "female"])
    feature_range_code(dtfm_ret, "Age", 10, 90, 10)      
    feature_one_code(dtfm_ret, "Pclass", [2, 3, 1])
    feature_one_code(dtfm_ret, "Embarked", ["S","C", "Q"])
    dtfm_ret.drop(["SibSp","Parch","Fare" ,"Embarked", "Pclass","Age","Sex"], axis = 1, inplace=True)
    return  dtfm_ret

train_feature = extract_features_4(df_train)
test_feature = extract_features_4(df_test)
train_target = train_feature.Survived
train_feature.drop(["Survived"], axis = 1, inplace = True)

test_target = test_feature.Survived
test_feature.drop(["Survived"], axis = 1, inplace = True)

evaluate(train_feature, test_feature, train_target, test_target)

[LibLinear]lr score 0.821989528796
-----
decision tree: 0.79057591623


# 取五维特征
## sex, age, Pclass， Embarked， Parch

In [7]:
def extract_features_5(dtfm):
    dtfm_ret = dtfm.drop(["PassengerId", "Name", "Cabin", "Ticket"], axis = 1)
    
    feature_one_code(dtfm_ret, "Sex", ["male", "female"])
    feature_range_code(dtfm_ret, "Age", 10, 90, 10)      
    feature_one_code(dtfm_ret, "Pclass", [2, 3, 1])
    feature_one_code(dtfm_ret, "Embarked", ["S","C", "Q"])
    feature_one_code(dtfm_ret, "Parch", [0, 1, 2, 6, 4, 3, 5])

    dtfm_ret.drop(["SibSp","Parch","Fare" ,"Embarked", "Pclass","Age","Sex"], axis = 1, inplace=True)
    return dtfm_ret

train_feature = extract_features_5(df_train)
test_feature = extract_features_5(df_test)
train_target = train_feature.Survived
train_feature.drop(["Survived"], axis = 1, inplace = True)

test_target = test_feature.Survived
test_feature.drop(["Survived"], axis = 1, inplace = True)

evaluate(train_feature, test_feature, train_target, test_target)

[LibLinear]lr score 0.806282722513
-----
decision tree: 0.753926701571


# 取六维特征
## sex, age, Pclass， Embarked， Parch，SibSp

In [8]:
def extract_features_6(dtfm):
    dtfm_ret = dtfm.drop(["PassengerId", "Name", "Cabin", "Ticket"], axis = 1)
    
    feature_one_code(dtfm_ret, "Sex", ["male", "female"])
    feature_range_code(dtfm_ret, "Age", 10, 90, 10)      
    feature_one_code(dtfm_ret, "Pclass", [2, 3, 1])
    feature_one_code(dtfm_ret, "Embarked", ["S","C", "Q"])
    feature_one_code(dtfm_ret, "Parch", [0, 1, 2, 6, 4, 3, 5])
    feature_one_code(dtfm_ret, "SibSp", [0, 2, 1, 3, 8, 4, 5])

    dtfm_ret.drop(["SibSp","Parch","Fare" ,"Embarked", "Pclass","Age","Sex"], axis = 1, inplace=True)
    return dtfm_ret

train_feature = extract_features_6(df_train)
test_feature = extract_features_6(df_test)
train_target = train_feature.Survived
train_feature.drop(["Survived"], axis = 1, inplace = True)

test_target = test_feature.Survived
test_feature.drop(["Survived"], axis = 1, inplace = True)

evaluate(train_feature, test_feature, train_target, test_target)

[LibLinear]lr score 0.806282722513
-----
decision tree: 0.79057591623
